# Text generation with GPT-2

Today we will try to generate texts using GPT-2 model proposed by OpenAI. This model is related to Transformer architecture (in fact, GPT-2 is a modified decoder from Transformers, similarly to BERT, which is a modification of an encoder). GPT-2 is a pretrained model that can be downloaded and used in the same way as BERT.

Here you can find a great introduction to the general idea behind GPT-2: https://jalammar.github.io/illustrated-gpt2/

In general, it is a language model, a model which provides us the probability of a given word being a continuation of a given text. For instance, having the following context: `Transformer is a neural network` GPT-2 can estimate that there is `50%` chance that the next word should be `architecture` and `0.0001%` chance that the next word is `donut`.

Let's use `Huggingface Transformers` library to experiment with GPT-2.

In [ ]:
!pip install transformers

# BASIC TEXT GENERATION

Let's start with the basic scenario -- as GPT-2 can calculate the probability of the next word given some context, it can be used to generate texts. In `transformers` library, we can do it pretty easily. `transformers` provides so-called pipelines, which hide all the abstraction layers so that we can generate texts using two lines of code.
It hides the: `Input -> Tokenization -> Model Inference -> Post-Processing (task dependent) -> Output` phases.


Please read the docs here: https://huggingface.co/docs/transformers/v4.19.2/en/main_classes/pipelines to familiarize with pipelines.

Then, fill the code below with appropriate fragments. In line 2, let's construct a pipeline of type `text-generation` and set the `model` parameter to `gpt2`.

Then, the `generator` can be called the same way as a function `generator(__some params here__)`. Just provide some first words of the text as string as a first positional argument (do not add a space at the end of it). You can provide additional parameters such as `max_length` (to limit the length of the generated text) or `num_return_sequences` (to force GPT-2 to produce multiple texts).

In [3]:
from transformers import pipeline
generator = pipeline("text-generation", model="gpt2")   # construct text-generation pipeline with model set to gpt2
generator("Alexander the Great once said", max_length = 30, num_return_sequences=2)      # uncomment and add parameters

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'Alexander the Great once said that it was a "miracle of God, that which is the great miracle of life". It is not only on earth'},
 {'generated_text': "Alexander the Great once said: 'This town is on the edge of destruction. If the British don't win the war they might go a long way"}]

There are various GPT-based models that are available in `transformers` library. Here: https://huggingface.co/models?search=gpt, you can find a list of them. They differ in the datasets they were trained on (the original GPT-2 was trained on Webtext https://paperswithcode.com/dataset/webtext, which consists of ~40GB of texts scraped from the internet) and the model sizes (e.g., GPT2-small consists of 117M parameters, GPT2-medium of 345M, GPT2-large of 762M).

Depending on our needs and available GPU memory, we can choose an appropriate one.
There are also distilled models that are `compressed` similarly to DistilBERT: https://huggingface.co/distilgpt2 (You can find more about distillation here: https://neptune.ai/blog/knowledge-distillation).

Check how models of different sizes relate to the quality of generated texts. Use `gpt2-small`, `gpt2-medium`, `gpt2-large` instead of `gpt2` in the pipeline and analyze the results.

Check how models trained on some more "specific" data work (e.g.,
`CodeGPT-small-java-adaptedGPT2` that can be used to write Java code)

*No report on the results is required. Just experiment if you are interested in this topic :)*


# GPT-2 as a source of knowledge
Since the model provides probable continuations of texts, we can use them to find answers to some questions.
You can type `The capital of Poland is` as a context to check if `Warsaw` will be proposed.

(beware!: don't add any whitespace at the end of the context. It frequently leads to some... strange results)

However, remember that the internet is biased. There is a lot of work that explores the bias of the GPT-models. This paper is an easy-to-follow analysis of the problem http://aclanthology.lst.uni-saarland.de/D19-1339.pdf. Because these models are trained on human-generated content, we should not treat them as oracles. Instead, we should treat them as some model of a stereotypical human being ;).

In [4]:
generator("A woman works as", max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'A woman works as a teacher at a Southland primary school in St. Catharines after Hurricane Irma sweeps past a school system center on May 19'},
 {'generated_text': "A woman works as a barber during a summer of snow in Tuxedo, Utah. She's part of a group of over 30 artists who"},
 {'generated_text': 'A woman works as a babysitter at a small home in the South of England which lies on the sea. A man works as a bouncer at'},
 {'generated_text': 'A woman works as a health care professional in the village of Khakkula. She has made two trips a week to receive the blood-s'},
 {'generated_text': 'A woman works as a taxi driver during a protest against what it calls the "illegal occupation" of the popular J-20 station in central Ankara in'}]

# Greedy search vs beam search

The default workflow of text generation with GPT-2 utilizes the greed search strategy. Given some context sequence, the model chooses a token with the highest probability as the continuation. However, in that scenario, we may generate "suboptimal" sequences. Please look at this webpage to grasp the idea of beam search https://huggingface.co/blog/how-to-generate. In short, beam search keeps the most likely `num_beams` of hypotheses at each time step and eventually chooses the hypothesis that has the overall highest probability.

The code below describes an alternative (let's call it classic) approach to using GPT. Instead of a pipeline, we generate the tokenizer and the model manually and then pass the tokenized context to the model. Please look at the call to `generate` function, you can find `num_beams` parameter which sets the number of beams to keep! Try to change it to see how the quality of the output changes.

In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

starting_context = "The GPT model is great"

input_ids = tokenizer(starting_context, return_tensors="pt").input_ids


outputs = gpt_model.generate(
    input_ids,
    num_beams=50,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The GPT model is great, but there's still a lot of work to be done.



## **Experimenting with different number of beams:** ##

For num_beams = 2: The GPT model is great, but it's not the best one. It doesn't have a

For num_beams = 5: The GPT model is great, but it's not perfect. It doesn't have all of the

For num_beams = 10: The GPT model is great, but there are a few things that need to be taken into account

For num_beams = 50: The GPT model is great, but there's still a lot of work to be done.

**Conclusion: The higher number of beams, the better generated sequence becomes**

# Constrained GPT-2

Sometimes we would like to constrain the output generated by the model. If you use the GPT2 model to write comments about your products, you want them to be positive :). Wouldn't it be useful to force GPT-2 to generate texts that have to contain some selected words like `wonderful`, `best` or `amazing`? :).

The GPT-2 models allow us to constrain the output in such a way. You can find a good introduction here: https://towardsdatascience.com/new-hugging-face-feature-constrained-beam-search-with-transformers-7ebcfc2d70e9
.

Analyze the snippet below (a modified code from the website mentioned above) to see how we can force GPT-2 to use some tokens. There are 2 cases:
* give some single token that has to be present somewhere in the generated text
* we give a list of alternatives from which the GPT-2 model chooses one.

Important sidenote: when experimenting with the code I once noticed that the model generated `besting` instead of the expected word `best`. I was surprised at first, but it works fine: while `best` is a token we expect to be present in the generated text, in transformer-related pretrained models, we use tokenization that may produce subword units. If, after `best` a continuation subtoken (e.g., `##ing` according to WordPiece notation that is used in BERT) is produced, then these tokens will be joined. That doesn't make the result wrong -- the token `best` is included in the generated text!

In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

must_contain_token = "best"
must_contain_alternatives = ["amazing", "wonderful", "beautiful", "exceptional"]  # let gpt choose which word to use


force_words_ids = [
    tokenizer([must_contain_token], add_prefix_space=True, add_special_tokens=False).input_ids,
    tokenizer(must_contain_alternatives, add_prefix_space=True, add_special_tokens=False).input_ids,
]

starting_text = ["The laptop", "The product"]
input_ids = tokenizer(starting_text, return_tensors="pt").input_ids


outputs = gpt_model.generate(
    input_ids,
    force_words_ids=force_words_ids,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(tokenizer.decode(outputs[1], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The laptop is powered by an Intel Core i7-4790K CPU, which has amazing best
The product is available in a variety of colors and sizes, including the standard black. The beautiful best



In 2020, a new version called GPT3 was created. While OpenAI didn't release the model providing only API-based access, some attempts to replicate the model are being made. You can find a model that should work in the same way as GPT3 here: https://huggingface.co/EleutherAI/gpt-neo-1.3B.
The story behind GPT3 and the reasons why it is not published as a downloadable model are described on Wikipedia: https://en.wikipedia.org/wiki/GPT-3.


## Other models, e.g., Gemma


Since 2019, many new models were proposed, many of them are much bigger than GPT-2, but some of them are of similar size. Check some new models, especially: `google/gemma-2-2b-it` and `HuggingFaceTB/SmolLM-1.7B` (there are even smaller versions of SmolLM so you can check them also/instead of this).

In [ ]:
from huggingface_hub import login

login("")

generator = pipeline("text-generation", model="google/gemma-2-2b-it")
txt1 = generator("Alexander the Great once said", truncation = True, max_length = 30)

generator = pipeline("text-generation", model="HuggingFaceTB/SmolLM-1.7B")
txt2 = generator("Alexander the Great once said", truncation = True, max_length = 30)

print(f"Gemma: {txt1}\nSmolLM: {txt2}")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/Kuba/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Gemma: [{'generated_text': 'Alexander the Great once said, "I am not afraid of death, for I know that I will be reborn in a new life." This quote'}]
SmolLM: [{'generated_text': 'Alexander the Great once said, “I am indebted to my father for living, but to my teacher for living well.”\nThe teacher is the one'}]


# vLLM

Please take a look at vLLM, which is an alternative to huggingface transformers. It is much faster, and production-wise, it may be a better choice. Still, it is far less popular than transformers, but it has the potential to become the state of the art solution soon.

Try to use vLLM to generate some LLM output.

Here are some useful links: https://docs.vllm.ai/en/latest/getting_started/examples/examples_index.html

https://github.com/vllm-project/vllm

In [16]:
from vllm import LLM, SamplingParams

llm = LLM(model="gpt2")
sampling_params = SamplingParams(temperature=0.7, top_p=0.9, max_tokens=20, stop=["happily", "forever"])

prompt = "What is the capital of Poland"
outputs = llm.generate(prompt, sampling_params)

print(outputs[0].outputs[0].text)

INFO 04-23 10:57:36 [config.py:2700] Downcasting torch.float32 to torch.float16.
INFO 04-23 10:57:36 [config.py:600] This model supports multiple tasks: {'generate', 'reward', 'score', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 04-23 10:57:36 [config.py:1634] Disabled the custom all-reduce kernel because it is not supported on current platform.
WARNING 04-23 10:57:36 [cpu.py:106] Environment variable VLLM_CPU_KVCACHE_SPACE (GiB) for CPU backend is not set, using 4 by default.
WARNING 04-23 10:57:36 [cpu.py:119] uni is not supported on CPU, fallback to mp distributed executor backend.
INFO 04-23 10:57:36 [llm_engine.py:242] Initializing a V0 LLM engine (v0.8.3) with config: model='gpt2', speculative_config=None, tokenizer='gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-23 10:57:37 [loader.py:447] Loading weights took 0.09 seconds
INFO 04-23 10:57:37 [executor_base.py:112] # cpu blocks: 7281, # CPU blocks: 0
INFO 04-23 10:57:37 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 113.77x
INFO 04-23 10:57:37 [llm_engine.py:448] init engine (profile, create kv cache, warmup model) took 0.42 seconds


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  6.50it/s, est. speed input: 39.09 toks/s, output: 130.30 toks/s]

?

The capital of Poland is Warsaw. The capital of Poland is Warsaw. The capital of
